In [46]:
import os

In [47]:
os.chdir('../')

In [48]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection'

In [56]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [57]:
from LLM_text_detection.utils.common import *
from LLM_text_detection.constants import *


In [58]:
read_yaml(CONFIG_FILE_PATH)

[2024-06-22 10:57:52,823: INFO: common: yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/Harshithvarma007/Datasets/raw/main/LLM-text-detection-dataset.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'unzip_data_dir': 'artifacts/data_ingestion/winequality-red.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/winequality-red.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_path': 'artifacts/data_transformation/train.csv', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_name': 'model.joblib'}, 'model_evaluation': {'root_dir': 'artifacts/model_evaluation', 'test_data_path': 'artifacts/data_transformation/test.csv', 'model_path': 'artifacts/model_trainer/model.joblib'

In [59]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [60]:
import os
import urllib.request as request
import zipfile
from LLM_text_detection import logger
from LLM_text_detection.utils.common import get_size

In [61]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [62]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-22 10:57:55,617: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-22 10:57:55,619: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-22 10:57:55,621: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-22 10:57:55,622: INFO: common: created directory at: artifacts]
[2024-06-22 10:57:55,623: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-22 10:58:01,542: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 31448525
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9032d941af047914d8687d4adc317732a757406822ccfc0b518289ffe1b386e0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BAFB:1650E0:43C74:60809:667660D9
Accept-Ranges: bytes
Date: Sat, 2